In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [15]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import pickle

In [6]:
df=pd.read_csv('data/test_geometric_features_normalized.csv')
# df2=pd.read_csv('geometric_features.csv')

linearity             740
planarity             740
sphericity            740
omnivariance          740
anisotropy            740
eigenentropy          740
sum_of_eigenvalues    740
change_curvature      740
id                    740
class                  12
reflectance            19
dtype: int64
anisotropy            2438
change_curvature      2438
eigenentropy          2438
linearity             2438
omnivariance          2438
planarity             2438
sphericity            2438
sum_of_eigenvalues    2438
class                   47
reflectance             37
dtype: int64


In [23]:
#testing saved ML models 

# scaler = StandardScaler()
X_test=pd.read_csv('../X_test.csv')
y_test=pd.read_csv('../y_test.csv')
y_test=y_test['true_labels'].to_numpy()
# X_test = scaler.transform(X_test)
X_test=(X_test-X_test.mean())/X_test.std()
# print(X_test.shape)
filename = 'knn.sav'
loaded_model = joblib.load(filename)
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
y_pred = loaded_model.predict(X_test)
print(y_pred)

AttributeError: 'numpy.ndarray' object has no attribute 'predict'

In [ ]:
#testing saved DL models
import os
import glob

import trimesh
from plyfile import PlyData, PlyElement
import open3d as o3d

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import callbacks

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, ConfusionMatrixDisplay,classification_report

import warnings
warnings.filterwarnings("ignore")

points = []
labels = []

for file in glob.glob('../data/sampled_ply/*.ply'):
# for file in glob.glob('paris_lille/sampled_ply/*.ply'):    
    
#     print(file)
    label = file.split('/')[-1].split('_')[2]
#     print(label)
    ply = PlyData.read(file)
    data = ply.elements[0].data
    data_pd = pd.DataFrame(data)
    data_np = np.zeros(data_pd.shape, dtype=np.float)
    property_names = data[0].dtype.names
    
    for i, name in enumerate(property_names):
        data_np[:,i] = data_pd[name]
        
    data_np = data_np[:,:3]    
    data_np = preprocessing.minmax_scale(data_np)
    
    points.append(data_np)
    labels.append(label)
object_classes = {
    '0' : 'Unclassified',
    '100000000' : 'Other',
    '200000000' : 'Surface',
    '201000000' : 'Other Surface',
    '202000000' : 'Ground',
    '202010000' : 'Other Ground',
    '202020000' : 'Road',
    '202030000' : 'Sidewalk',
    '202040000' : 'Curb',
    '202050000' : 'Island',
    '202060000' : 'Vegetation',
    '203000000' : 'Building',
    '300000000' : 'Object',
    '301000000' : 'Other Object',
    '302000000' : 'Static',
    '302010000' : 'Other Static',
    '302020000' : 'Punctual Object',
    '302020100' : 'Other Punctual Object',
    '302020200' : 'Post',
    '302020300' : 'Bollard',
    '302020400' : 'Floor Lamp',
    '302020500' : 'Traffic Light',
    '302020600' : 'Traffic Sign',
    '302020700' : 'Signboard',
    '302020800' : 'Mailbox',
    '302020900' : 'Trash Can',
    '302021000' : 'Meter',
    '302021100' : 'Bicycle Terminal',
    '302021200' : 'Bicycle Rack',
    '302021300' : 'Statue',
    '302030000' : 'Linear',
    '302030100' : 'Other Linear',
    '302030200' : 'Barrier',
    '302030300' : 'Roasting',
    '302030400' : 'Grid',
    '302030500' : 'Chain',
    '302030600' : 'Wire',
    '302030700' : 'Low Wall',
    '302040000' : 'Extended',
    '302040100' : 'Other Extended',
    '302040200' : 'Shelter',
    '302040300' : 'Kiosk',
    '302040400' : 'Scaffold',
    '302040500' : 'Bench',
    '302040600' : 'Distribution Box',
    '302040700' : 'Lighting Console',
    '302040800' : 'Windmill',
    '303000000' : 'Dynamic',
    '303010000' : 'Other Dynamic',
    '303020000' : 'Pedestrian',
    '303020100' : 'Other Pedestrian',
    '303020200' : 'Still Pedestrian',
    '303020300' : 'Walking Pedestrian',
    '303020400' : 'Running Pedestrian',
    '303020500' : 'Stroller Pedestrian',
    '303020600' : 'Holding Pedesterian',
    '303020700' : 'Leaning Pedestrian',
    '303020800' : 'Skater',
    '303020900' : 'Rollerskater',
    '303021000' : 'Wheelchair',
    '303030000' : '2 Wheelers',
    '303030100' : 'Other 2 Wheels',
    '303030200' : 'Bicycle',
    '303030201' : 'Other Bicycle',
    '303030202' : 'Mobile Bicycle',
    '303030203' : 'Stopped Bicycle',
    '303030204' : 'Parked Bicycle',
    '303030300' : 'Scooter',
    '303030301' : 'Other Scooter',
    '303030302' : 'Mobile Scooter',
    '303030303' : 'Stopped Scooter',
    '303030304' : 'Parked Scooter',
    '303030400' : 'Moped',
    '303030401' : 'Other Moped',
    '303030402' : 'Mobile Moped',
    '303030403' : 'Stopped Moped',
    '303030404' : 'Parked Moped',
    '303030500' : 'Motorbike',
    '303030501' : 'Other Motorbike',
    '303030502' : 'Mobile Motorbike',
    '303030503' : 'Stopped Motorbike',
    '303030504' : 'Parked Motorbike',
    '303040000' : '4+ Wheelers',
    '303040100' : 'Other 4+ Wheelers',
    '303040200' : 'Car',
    '303040201' : 'Other Car',
    '303040202' : 'Mobile Car',
    '303040203' : 'Stopped Car',
    '303040204' : 'Parked Car',
    '303040300' : 'Van',
    '303040301' : 'Other Van',
    '303040302' : 'Mobile Van',
    '303040303' : 'Stopped Van',
    '303040304' : 'Parked Van',
    '303040400' : 'Truck',
    '303040401' : 'Other Truck',
    '303040402' : 'Mobile Truck',
    '303040403' : 'Stopped Truck',
    '303040404' : 'Parked Truck',
    '303040500' : 'Bus',
    '303040501' : 'Other Bus',
    '303040502' : 'Mobile Bus',
    '303040503' : 'Stopped Bus',
    '303040504' : 'Parked Bus',
    '303050000' : 'Furniture',
    '303050100' : 'Other Furniture',
    '303050200' : 'Table',
    '303050300' : 'Chair',
    '303050400' : 'Stool',
    '303050500' : 'Trash Can',
    '303050600' : 'Waste',
    '304000000' : 'Natural',
    '304010000' : 'Other Natural',
    '304020000' : 'Tree',
    '304030000' : 'Bush',
    '304040000' : 'Potted Plant',
    '304050000' : 'Hedge'
}


# In[64]:


labels = [object_classes.get(item,item) for item in labels]
unique_labels = np.unique(labels)

mapping = {}
keys = range(len(unique_labels))

for i in keys:
    mapping[i] = unique_labels[i]
    
rev_mapping = dict((v,k) for k,v in mapping.items())
new_labels = [rev_mapping[i] for i in labels]


# In[102]:


## testing KP
# print(keys)
# print(unique_labels[0])
# print(rev_mapping)
# print(new_labels)
# print(labels)


# In[66]:


NUM_POINTS = 1000
NUM_CLASSES = 48
BATCH_SIZE = 32


# In[67]:


# checking if gpu is available or not KP
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)


# In[68]:


X_train, X_test, y_train, y_test = train_test_split(points, new_labels, test_size=.33, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.15, random_state=0)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)


model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [4]:
import os
import glob

import multiprocessing as mp
import time

def check(file):
    print('file: ',file)
    
#     return num*2
ply_dir='../data/test_ply_objects/' 
input_path = os.path.join(ply_dir, '*.ply')
num_cores = mp.cpu_count()
pool = mp.Pool(num_cores)
# print(num_cores)
# start = time.time()
pool.map(check, glob.glob(input_path))
# pool.map(check, [1,2,3,4])
# end = time.time()
# print(end - start)

file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file: file:                        ../data/test_ply_objects/228-302040700_544.ply../data/test_ply_objects/147-302020700_400.ply../data/test_ply_objects/1414-202060000_900.ply../data/test_ply_objects/271-304020000_385.ply../data/test_ply_objects/10-304000000_17.ply../data/test_ply_objects/2268-303050500_250.ply../data/test_ply_objects/728-302020400_841.ply../data/test_ply_objects/1547-302040700_1312.ply../data/test_ply_objects/1906-303020000_401.ply../data/test_ply_objects/1274-302020300_912.ply../data/test_ply_objects/241-302020400_305.ply../data/test_ply_objects/1053-302030600_1078.ply../data/test_ply_objects/2320-304020000_124.ply../data/test_ply_objects/1397-303040202_312.ply../data/test_ply_objects/2306-3040200

../data/test_ply_objects/1471-303040204_547.ply   file: file: 
../data/test_ply_objects/589-302021200_549.ply../data/test_ply_objects/234-302020600_490.ply../data/test_ply_objects/1061-303040204_254.ply
../data/test_ply_objects/2345-302020400_193.ply
../data/test_ply_objects/1656-302020300_1144.ply../data/test_ply_objects/350-304020000_141.ply../data/test_ply_objects/262-303020000_328.plyfile:  ../data/test_ply_objects/1044-302030600_1348.ply../data/test_ply_objects/2141-303050500_307.ply
file: file: file: 

file: 
 
file: 
file: ../data/test_ply_objects/1548-303040304_570.ply
../data/test_ply_objects/2274-303050500_336.plyfile: file:  file:        



../data/test_ply_objects/178-303030304_355.ply
../data/test_ply_objects/546-304020000_90.ply../data/test_ply_objects/159-302040700_547.ply 
 ../data/test_ply_objects/1780-302020300_1025.ply

file: 



../data/test_ply_objects/1575-303050500_1109.plyfile: file:  file:  file: file: file: file: file:    ../data/test_ply_objects/119-30202030

file: ../data/test_ply_objects/411-303030304_438.plyfile:   file: ../data/test_ply_objects/1071-302040700_1334.ply   file: ../data/test_ply_objects/1759-302020300_1149.ply../data/test_ply_objects/972-304020000_246.ply   file: 
 ../data/test_ply_objects/320-304020000_56.ply   ../data/test_ply_objects/1489-304000000_520.ply  
file:    ../data/test_ply_objects/1631-302030300_63.ply ../data/test_ply_objects/994-303040204_376.plyfile:  
file: 
 file: file: ../data/test_ply_objects/1971-304000000_79.ply
file: ../data/test_ply_objects/1400-303040204_207.ply
../data/test_ply_objects/1585-302020600_730.ply../data/test_ply_objects/1920-302040700_1341.plyfile: 
../data/test_ply_objects/574-303040304_68.ply../data/test_ply_objects/1541-303040404_347.ply
../data/test_ply_objects/1360-302040600_380.ply../data/test_ply_objects/1963-302020500_133.ply file: ../data/test_ply_objects/750-302020200_668.ply../data/test_ply_objects/8-302030300_7.ply../data/test_ply_objects/1181-302030700_768.ply../data/test

file:  file: ../data/test_ply_objects/1424-303040204_168.ply   ../data/test_ply_objects/1676-302020600_833.ply file:    file: file: file: 
 file: ../data/test_ply_objects/52-303050300_466.plyfile: 
 ../data/test_ply_objects/911-302021100_1151.ply../data/test_ply_objects/2377-304000000_339.ply
 ../data/test_ply_objects/1319-303040204_163.ply   ../data/test_ply_objects/583-303040304_42.plyfile:  ../data/test_ply_objects/998-202060000_97.plyfile: file:  ../data/test_ply_objects/2123-302040700_453.ply  file: 
 ../data/test_ply_objects/423-302020300_530.plyfile: ../data/test_ply_objects/1761-203000000_3.plyfile: ../data/test_ply_objects/1551-302020400_475.ply
file: ../data/test_ply_objects/613-302021200_533.ply file: 
../data/test_ply_objects/380-304020000_322.ply


../data/test_ply_objects/1264-304000000_915.ply../data/test_ply_objects/1079-304020000_295.ply../data/test_ply_objects/2130-302040700_482.ply
   
file: 
../data/test_ply_objects/61-203000000_13.ply file:   
 ../data/test_ply_obj

../data/test_ply_objects/2122-302040700_479.ply ../data/test_ply_objects/2304-302020400_277.ply ../data/test_ply_objects/151-302020300_592.ply 

file: file: 
 

../data/test_ply_objects/1006-303040204_378.ply
file:  file:   ../data/test_ply_objects/1762-302030600_1066.ply../data/test_ply_objects/551-302020700_260.ply
file: file:  file: 
 file: ../data/test_ply_objects/1002-302030200_677.ply../data/test_ply_objects/1514-304040000_615.ply 
file: file: 

../data/test_ply_objects/2398-203000000_16.ply 

 
../data/test_ply_objects/616-302020900_426.ply
../data/test_ply_objects/1651-302020300_1142.ply ../data/test_ply_objects/1379-302020300_1108.ply../data/test_ply_objects/2430-302020400_293.ply../data/test_ply_objects/2258-303050500_236.ply ../data/test_ply_objects/1389-302020400_950.ply
../data/test_ply_objects/725-302020600_972.ply file: ../data/test_ply_objects/2428-202060000_29.ply
file: 
 ../data/test_ply_objects/605-302021200_497.ply  
file: 

 
file: file: 
file: file: ../data/test_p

...

  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_send, copy=copy)
  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_send, copy=copy)
  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_send, copy=copy)
  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_send, copy=copy)
  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_send, copy=copy)
  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_send, copy=copy)
  File "/ext3/miniconda3/lib/python3.9/site-packages/jupyter_client/session.py", line 853, in send
    stream.send_multipart(to_se

  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in send_multipart
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in send_multipart
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in send_multipart
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 216, in schedule
    f()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in send_multipart
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 216, in schedule
    f()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 216, in schedule
    f()
  File 

  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in <lambda>
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in <lambda>
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in <lambda>
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in <lambda>
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 223, in <lambda>
    self.schedule(lambda : self._really_send(*args, **kwargs))
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.

  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/zmq/sugar/context.py", line 190, in term
    return super(Context, self).term()
  File "/ext3/miniconda3/lib/python3.9/site-packages/ipykernel/iostream.py", line 239, in _really_send
    ctx.term()
  File "/ext3/miniconda3/lib/python3.9/si

  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
KeyboardInterrupt
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/context.pyx", line 82, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.b

In [5]:
import os
import glob

import multiprocessing as mp
import time

import numpy as np
import pandas as pd

from plyfile import PlyData, PlyElement
import open3d as o3d
import trimesh

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# def createBPTriangleMesh(directory):
    
#     for file in glob.glob(directory):
    
#         pcd = o3d.io.read_point_cloud(file)

#         if len(np.asarray(pcd.points)) < 250:
#             continue

#         pcd.estimate_normals()

#         distances = pcd.compute_nearest_neighbor_distance()
#         avg_distance = np.mean(distances)
#         radius = 1.5 * avg_distance

#         mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius*2]))
#         filename = '../data/test_mesh_objects/' + 'mesh_' + file.split('/')[-1]
#         #to save to a file
#         o3d.io.write_triangle_mesh(filename, mesh)

#         sampled = mesh.sample_points_poisson_disk(number_of_points=1000)
#         filename = '../data/test_sampled_ply/' + 'sampled_' + file.split('/')[-1]
#         #to save to a file
#         o3d.io.write_point_cloud(filename, sampled)

        
        
def createBPTriangleMesh(file):
    print('on it')
    pcd = o3d.io.read_point_cloud(file)

    if len(np.asarray(pcd.points)) > 250:
#         print('creating file')
        pcd.estimate_normals()

        distances = pcd.compute_nearest_neighbor_distance()
        avg_distance = np.mean(distances)
        radius = 1.5 * avg_distance

        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector([radius, radius*2]))
        filename = '../data/test_mesh_objects/' + 'mesh_' + file.split('/')[-1]
        #to save to a file
        o3d.io.write_triangle_mesh(filename, mesh)
        
        print('working on ',filename)

        sampled = mesh.sample_points_poisson_disk(number_of_points=1000)
        filename = '../data/test_sampled_ply/' + 'sampled_' + file.split('/')[-1]
        #to save to a file
        o3d.io.write_point_cloud(filename, sampled)
    else:
        pass
        
        
        
# ply_dir='../data/test_ply_objects/'     
# ply_dir = '/home/jupyter-seanandrewchen/shared/cusp-capstone/data/paris_lille/ply_objects/'
# input_path = os.path.join(ply_dir, '*.ply')
p1='../data/test_ply_objects/1000-302020600_700.ply'
p2='../data/test_ply_objects/1007-304020000_505.ply'
p3='../data/test_ply_objects/1010-303040204_555.ply'
p4='../data/test_ply_objects/1047-202050000_451.ply'
# createBPTriangleMesh(p1)
num_cores = mp.cpu_count()
pool = mp.Pool(num_cores)
print(num_cores)
start = time.time()
# # pool.map(createBPTriangleMesh, glob.glob(input_path))
pool.map(createBPTriangleMesh, [p1,p2,p3,p4])
end = time.time()
print(end - start)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
on iton iton iton it



48


Process ForkPoolWorker-284:
Process ForkPoolWorker-273:
Process ForkPoolWorker-274:
Process ForkPoolWorker-260:
Process ForkPoolWorker-288:
Process ForkPoolWorker-267:
Process ForkPoolWorker-256:
Process ForkPoolWorker-281:
Process ForkPoolWorker-283:
Process ForkPoolWorker-278:
Process ForkPoolWorker-286:
Process ForkPoolWorker-262:
Process ForkPoolWorker-271:
Process ForkPoolWorker-246:
Process ForkPoolWorker-258:
Process ForkPoolWorker-275:
Process ForkPoolWorker-268:
Process ForkPoolWorker-247:
Process ForkPoolWorker-265:
Process ForkPoolWorker-252:
Process ForkPoolWorker-251:
Process ForkPoolWorker-282:
Process ForkPoolWorker-249:
Process ForkPoolWorker-292:
Process ForkPoolWorker-289:
Process ForkPoolWorker-291:
Process ForkPoolWorker-290:
Process ForkPoolWorker-264:
Process ForkPoolWorker-248:
Process ForkPoolWorker-279:
Process ForkPoolWorker-259:
Process ForkPoolWorker-250:
Process ForkPoolWorker-272:
Process ForkPoolWorker-280:
Process ForkPoolWorker-253:
Process ForkPoolWork

  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in ru

  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/pool.py", line 114, in work

  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/synchronize.py", line 95, 

KeyboardInterrupt: 

KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/ext3/miniconda3/lib/python3.9/multiprocessing/connection.py", line 384, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [8]:
s='../data/test_ply_objects/123-12324123_123.ply'
print(s.split('/')[-1].split('-')[0])
print(s.split('/')[-1].split('-')[-1])

123
12324123_123.ply
